In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_csv('data/TrainingData.csv')

In [9]:
df.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [25]:
df.set_index('Id', inplace=True)

In [26]:
df.nunique()

Income               41920
Age                     59
Experience              21
Married/Single           2
House_Ownership          3
Car_Ownership            2
Profession              51
CITY                   317
STATE                   29
CURRENT_JOB_YRS         15
CURRENT_HOUSE_YRS        5
Risk_Flag                2
dtype: int64

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252000 entries, 1 to 252000
Data columns (total 12 columns):
Income               252000 non-null int64
Age                  252000 non-null int64
Experience           252000 non-null int64
Married/Single       252000 non-null object
House_Ownership      252000 non-null object
Car_Ownership        252000 non-null object
Profession           252000 non-null object
CITY                 252000 non-null object
STATE                252000 non-null object
CURRENT_JOB_YRS      252000 non-null int64
CURRENT_HOUSE_YRS    252000 non-null int64
Risk_Flag            252000 non-null int64
dtypes: int64(6), object(6)
memory usage: 25.0+ MB


In [31]:
numerical_features = list(df.loc[:, df.dtypes == 'int64'].drop('Risk_Flag', axis=1).columns)
numerical_features

['Income', 'Age', 'Experience', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS']

In [32]:
categorical_features = list(df.loc[:, df.dtypes=='object'].columns)
categorical_features

['Married/Single',
 'House_Ownership',
 'Car_Ownership',
 'Profession',
 'CITY',
 'STATE']

In [30]:
from sklearn.feature_selection import f_classif

In [33]:
f_values = f_classif(df[numerical_features], df['Risk_Flag'])

In [34]:
f_values

(array([  2.40794463, 119.9185068 , 300.69230652,  72.34852625,
          4.82385115]),
 array([1.20722032e-01, 6.68762365e-28, 2.54756349e-67, 1.81322122e-17,
        2.80694728e-02]))

In [36]:
f_values[1]

array([1.20722032e-01, 6.68762365e-28, 2.54756349e-67, 1.81322122e-17,
       2.80694728e-02])

In [37]:
p_values = pd.Series(index=numerical_features, data=f_values[1])
p_values

Income               1.207220e-01
Age                  6.687624e-28
Experience           2.547563e-67
CURRENT_JOB_YRS      1.813221e-17
CURRENT_HOUSE_YRS    2.806947e-02
dtype: float64

In [38]:
from sklearn.feature_selection import mutual_info_classif

In [39]:
mi = mutual_info_classif(df[categorical_features], df['Risk_Flag'])

ValueError: could not convert string to float: 'single'